In [1]:
import sys
sys.path.append("../src")
sys.path.append("../src/models")
import sklearn
import tslearn
import argparse
from train import prepare_dataset
from experiments import experiments
import tqdm
import numpy as np
from tslearn.svm import TimeSeriesSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from utils.classmetric import confusion_matrix_to_accuraccies
from sklearn.metrics import confusion_matrix

from tslearn.utils import to_time_series_dataset
from time import time
from sklearn.model_selection import RandomizedSearchCV
import scipy
from sklearn.ensemble import RandomForestClassifier

from utils.data2numpy import get_data

def flatten(x):
    return x.reshape(x.shape[0], -1)

In [11]:
N_per_class = 500

tum_ds = get_data("isprs_rf_tum_23classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)
gaf_ds = get_data("isprs_rf_gaf_23classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)
tum12_ds = get_data("isprs_rf_tum_12classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)
gaf12_ds = get_data("isprs_rf_gaf_12classes", N_per_class=N_per_class, N_largest=None, do_add_spectral_indices=True)


Initializing BavarianCropsDataset test partition in holl
read 23 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs.csv/blocks/holl/test
loaded 9792 samples
Dataset /data/BavarianCrops. region holl. partition test. X:9792x(144, 13), y:(9792,) with 23 classes
Initializing BavarianCropsDataset test partition in nowa
read 23 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs.csv/blocks/nowa/test
loaded 3572 samples
Dataset /data/BavarianCrops. region nowa. partition test. X:3572x(287, 13), y:(3572,) with 23 classes
Initializing BavarianCropsDataset test partition in krum
read 23 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs.csv/blocks/krum/test
loaded 4306 samples
Dataset /data/BavarianCrops. region krum. partition test. X:4306x(143, 13), y:(4306,) with 23 classes
Initializing BavarianCropsDataset trainvalid partition in holl
read 23 classes
precached dataset files found at /data/Bavari

7it [00:00, 68.89it/s]

loaded 25368 samples
Dataset /data/BavarianCrops. region krum. partition trainvalid. X:25368x(71, 13), y:(25368,) with 23 classes
setting random seed to 0


232it [00:03, 67.64it/s]
70it [00:00, 71.27it/s]


saving npy arrays to /data/GAFdataset/npy/holl
features='optical': selecting 10 optical features from all 18 features
gaf classes [6, 17, 21, 22, 23, 34, 45, 60, 102, 105] not in classmapping /data/BavarianCrops/classmapping.isprs.csv. ignoring 91 examples (1.26% of dataset)
Dataset /data/GAFdataset/test_train_holl.h5. region holl. partition test. X:(7117, 23, 10), y:(7117,) with 23 classes
saving npy arrays to /data/GAFdataset/npy/nowa


...

228it [00:03, 72.86it/s]
68it [00:01, 63.61it/s]


saving npy arrays to /data/GAFdataset/npy/holl
features='optical': selecting 10 optical features from all 18 features
gaf classes [6, 7, 13, 15, 17, 21, 22, 23, 34, 36, 37, 45, 60, 102, 105] not in classmapping /data/BavarianCrops/classmapping.isprs2.csv. ignoring 190 examples (2.64% of dataset)
Dataset /data/GAFdataset/test_train_holl.h5. region holl. partition test. X:(7018, 23, 10), y:(7018,) with 12 classes
saving npy arrays to /data/GAFdataset/npy/nowa
features='optical': selecting 10 optical features from all 18 features
gaf classes [13] not in classmapping /data/BavarianCrops/classmapping.isprs2.csv. ignoring 1 examples (0.04% of dataset)
Dataset /data/GAFdataset/test_train_nowa.h5. region nowa. partition test. X:(2659, 23, 10), y:(2659,) with 12 classes
saving npy arrays to /data/GAFdataset/npy/krum
features='optical': selecting 10 optical features from all 18 features
gaf classes [5, 6, 7, 13, 15, 22, 34, 36, 37, 42, 47, 50, 60, 83, 91] not in classmapping /data/BavarianCrops/

0it [00:00, ?it/s]

features='optical': selecting 10 optical features from all 18 features
gaf classes [5, 6, 7, 13, 15, 22, 34, 36, 37, 42, 47, 50, 60, 83, 91] not in classmapping /data/BavarianCrops/classmapping.isprs2.csv. ignoring 339 examples (1.93% of dataset)
Dataset /data/GAFdataset/test_train_krum.h5. region krum. partition train. X:(17267, 23, 10), y:(17267,) with 12 classes
setting random seed to 0


150it [00:17,  8.76it/s]
53it [00:05,  8.92it/s]


# Support Vector Machine

## TUM dataset

In [ ]:
!cat /data/isprs/sklearn/svm_tum.txt
#svm_tum_params = {'C': 34.94762791972138, 'gamma': 0.09632187442535493, 'kernel': 'rbf'}
#svm_tum_params ={'C': 219.5746177088989, 'gamma': 0.030512293944411368, 'kernel': 'rbf'}
svm_tum_params ={'C': 219.5746177088989, 'gamma': 0.030512293944411368, 'kernel': 'rbf'}
!cat /data/isprs/sklearn/svm_gaf.txt
svm_gaf_params = {'C': 83.60975452306279, 'gamma': 0.03084148398718425, 'kernel': 'rbf'}

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum_ds

clf = TimeSeriesSVC(n_jobs=8,**svm_tum_params)
clf.fit(X, y)
y_pred = clf.predict(Xtest)
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

clf = sklearn.svm.SVC(**svm_tum_params)
#clf = TimeSeriesSVC(n_jobs=8,**svm_tum_params)
clf.fit(feat_ext(X), y)
y_pred = clf.predict(feat_ext(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

In [ ]:
!pip install tsfresh
from tsfresh import extract_features
extracted_features = extract_features(X[0,:,0])

In [ ]:
def feat_ext(X):
    f1 = X.mean(1)
    f2 = X.std(1)
    f3 = np.median(X,1)
    return np.hstack([f1,f2,f2])

## GAF Dataset

In [ ]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

clf = TimeSeriesSVC(n_jobs=8,**svm_gaf_params)
clf.fit(X, y)
y_pred = clf.predict(Xtest)
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
params = {'n_estimators': 1400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': False}


In [4]:
!cat /data/isprs/sklearn/random_forest_tum.txt
rf_tum_params = {'bootstrap': False, 'max_depth': 70, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 2000}
#rf_tum_params = {'bootstrap': False, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 2000}
!cat /data/isprs/sklearn/random_forest_gaf.txt
rf_gaf_params = {'bootstrap': False, 'max_depth': 60, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 1000}
#rf_gaf_params = {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1200}

{'bootstrap': False, 'max_depth': 70, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 2000} score: 0.4562794154074808
{'bootstrap': False, 'max_depth': 60, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 1000} score: 0.6899191833262441


## Raw Dataset 23 classes

In [12]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                        precision    recall  f1-score   support

                fallow       0.23      0.15      0.19       356
         fallow+flower       0.29      0.30      0.29       225
               alfalfa       0.50      0.14      0.22       500
             grassland       0.31      0.13      0.19       469
         proteinplants       0.00      0.00      0.00        23
          winter wheat       0.38      0.18      0.24       500
                  corn       0.72      0.72      0.72       500
          summer wheat       0.00      0.00      0.00        56
              beetroot       1.00      0.06      0.12        16
               potatoe       0.09      0.22      0.12       121
   grassland+machining       0.29      0.34      0.32       500
      grassland+cattle       0.15      0.29      0.19       226
          winter spelt       1.00      0.02      0.04        52
            winter rye       0.33      0.01      0.02       104
         winter barley       0.49      

/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Preprocessed Dataset 23 classes

In [13]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_gaf_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                        precision    recall  f1-score   support

                fallow       0.31      0.35      0.33       136
         fallow+flower       0.69      0.75      0.72       159
               alfalfa       0.63      0.41      0.49       500
             grassland       0.33      0.31      0.32       314
         proteinplants       0.00      0.00      0.00        21
          winter wheat       0.63      0.52      0.57       500
                  corn       0.99      0.98      0.98       500
          summer wheat       1.00      0.12      0.22        33
              beetroot       1.00      0.30      0.46        10
               potatoe       0.20      0.40      0.27         5
   grassland+machining       0.53      0.42      0.47       500
      grassland+cattle       0.24      0.66      0.35       153
          winter spelt       0.60      0.15      0.24        41
            winter rye       0.58      0.11      0.19        97
         winter barley       0.82      

/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## raw Dataset 12 classes

In [14]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = tum12_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                  precision    recall  f1-score   support

          fallow       0.53      0.38      0.45       500
       grassland       0.63      0.56      0.60       500
    winter wheat       0.42      0.21      0.28       500
            corn       0.82      0.79      0.80       500
    summer wheat       0.00      0.00      0.00        56
    winter spelt       0.00      0.00      0.00        52
      winter rye       0.22      0.02      0.04       104
   winter barley       0.52      0.26      0.35       500
   summer barley       0.53      0.48      0.51       500
      summer oat       0.25      0.59      0.35       275
winter triticale       0.25      0.56      0.35       499
        rapeseed       0.79      0.73      0.76       257

        accuracy                           0.47      4243
       macro avg       0.41      0.38      0.37      4243
    weighted avg       0.51      0.47      0.46      4243

overall accuracy = 0.47, kappa = 0.40, precision = 0.41, recall=0.38,

/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## pre Dataset 12 classes

In [15]:
X,y,ids, Xtest, ytest, idstest, classnames, class_idxs = gaf12_ds

rf_tum = RandomForestClassifier(n_jobs=-1,**rf_tum_params)
rf_tum.fit(flatten(X),y)
y_pred = rf_tum.predict(flatten(Xtest))
print(classification_report(ytest, y_pred,labels=class_idxs, target_names=classnames))
metrics = confusion_matrix_to_accuraccies(confusion_matrix(ytest,y_pred))
overall_accuracy, kappa, precision, recall, f1, cl_acc = metrics
print(f"overall accuracy = {overall_accuracy:.2f}, kappa = {kappa:.2f}, precision = {precision.mean():.2f},",
      f"recall={recall.mean():.2f}, f1={f1.mean():.2f}")

                  precision    recall  f1-score   support

          fallow       0.66      0.87      0.75       295
       grassland       0.93      0.79      0.85       500
    winter wheat       0.68      0.50      0.57       500
            corn       1.00      0.98      0.99       500
    summer wheat       1.00      0.15      0.26        33
    winter spelt       0.64      0.17      0.27        41
      winter rye       0.37      0.13      0.20        97
   winter barley       0.80      0.71      0.75       500
   summer barley       0.70      0.89      0.79       500
      summer oat       0.62      0.74      0.68       220
winter triticale       0.51      0.66      0.58       440
        rapeseed       0.89      0.91      0.90       248

        accuracy                           0.75      3874
       macro avg       0.73      0.62      0.63      3874
    weighted avg       0.76      0.75      0.74      3874

overall accuracy = 0.75, kappa = 0.71, precision = 0.73, recall=0.62,